# Demonstration of pyQ3 python wrapper for DEW-ENKI-EQ3 aqueous speciation calculations - TUTORIAL 3
**Important: before running any calculations you must have a compiled set of DEW species objects. If you are running this on your own machine you should run the import_DEW.ipynb notebook before running this one.**

First import the required packages:

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pyQ3

## Calculate fluid composition and speciation for a fluid in equilibrium with a rock

More realistic calculations involve equilibrium with minerals. Every element in the system that is not directly specified by molality must be specified by equilibrium with a mineral or solid solution. fO2 and pH are set to a default value or by charge balance unless directly specified.

### Quartz solubility in brine

Create an instance of the default system

In [3]:
dew_system = pyQ3.System()

Set up the elements of the fluid not constrained by mineral equilibria:

In [5]:
molalities = {'NA+':0.05,
              'CL-':0.05,
              }
T = 300.0 + 273.15 # in K
P = 10000.0 # in bar

Tell EQ3 to set the Si content of the solution by equilibrium with Quartz:

In [6]:
mineral_eq = {'SIO2(AQ)':'QUARTZ'}

Create the fluid:

In [7]:
dew_fluid = pyQ3.Fluid(dew_system,T,P,molalities=molalities,mineral_eq=mineral_eq)

First we can check that quartz is the stable mineral phase at these conditions:

In [8]:
dew_fluid.mineral_saturation

,mineral,log_q_k,aff,state
0,QUARTZ,0.000,0.000,satd
1,COESITE,-0.322,-0.845,NaN


Coesite is not stable, and there are no other SiO2 minerals in the system. Let's look at the dissolved Si content of the solution:

In [10]:
dew_fluid.elemental_comp

,element,ppm,molality
0,O,889218.8788,.5557826411E+02
1,H,111963.3722,.1110857944E+03
2,SI,496.6487,.1768345440E-01
3,CL,1772.6500,.5000000000E-01
4,NA,1149.4885,.5000000000E-01


But in what species is the Si dissolved?

In [11]:
dew_fluid.aqueous_species

,species,molality,log_g,activity
0,NA+,4.955004e-02,-0.0921,4.008000e-02
1,CL-,4.954867e-02,-0.0921,4.008000e-02
2,SIO2(AQ),1.524053e-02,0.0000,1.524000e-02
3,SI2O4(AQ),1.208975e-03,0.0000,1.209000e-03
4,NACL(AQ),4.487682e-04,0.0000,4.488000e-04
5,H+,8.011735e-05,-0.0921,6.481000e-05
6,OH-,5.651803e-05,-0.0921,4.572000e-05
7,HSIO3-,2.497210e-05,-0.0921,2.020000e-05
8,HCL(AQ),2.566052e-06,0.0000,2.566000e-06
9,NAOH(AQ),1.193273e-06,0.0000,1.193000e-06


And what's the pH?

In [12]:
dew_fluid.pH

4.1884

### A fluid in equilibrium with Fayalite-Magnetite-Quartz fO2 buffer

In this example we will allow the fO2 of the fluid to be set by equilibrium with a mineral assemblage. The system involves both Fe and Si, so both of these must also be set:

In [13]:
mineral_eq = {'SIO2(AQ)':'QUARTZ',
              'FE+2':'FAYALITE',
              'O2':'MAGNETITE'}
T = 300.0 + 273.15 # in K
P = 10000.0 # in bar

This time we won't add any NaCl.

In [14]:
dew_fluid = pyQ3.Fluid(dew_system, T, P, mineral_eq = mineral_eq)

Hopefully the code has correctly identified that the fO2 is being set by mineral equilibria, and so it hasn't used the abitrary value, let's check:

In [15]:
dew_fluid.fO2

-34.525

Let's check if any other minerals are saturated:

In [16]:
dew_fluid.mineral_saturation

,mineral,log_q_k,aff,state
0,MAGNETITE,0.000,0.000,satd
1,HEMATITE,-0.775,-2.031,NaN
2,FAYALITE,0.000,0.000,satd
3,FERROSILITE,-0.015,-0.039,satd
4,QUARTZ,0.000,0.000,satd
5,COESITE,-0.322,-0.845,NaN


Ferrosilite is saturated. Perhaps this isn't a surprise that the FMQ buffer isn't stable everywhere, but is instead a useful reference point...

### Fluids in equilibrium with solid solutions

We can see what the chemistry of a fluid in equilibrium with a fictional harzburgite. First let's see what the default solid solutions built into the module are:

In [17]:
pyQ3.defaultsystem.solid_solutions

{'K-FELDSPAR(SS)': ['POTASSIUM_FEL', 'ALBITE'],
 'PLAGIOCLASE(SS)': ['ALBITE', 'ANORTHITE'],
 'ORTHOPYROXENE(SS)': ['ORTHOENSTATIT', 'FERROSILITE'],
 'OLIVINE(SS)': ['FORSTERITE', 'FAYALITE'],
 'GARNET(SS)': ['PYROPE', 'ALMANDINE', 'GROSSULAR'],
 'CLINOPYROXENE(SS)': ['DIOPSIDE', 'JADEITE']}

Similarly to the pure mineral equilibria, we set the species by equilibrium in the mineral_eq variable, but this time we use a list containing the name of the solid solution, and the name of the endmember.

In [33]:
mineral_eq = {'SIO2(AQ)': ['ORTHOPYROXENE(SS)','ORTHOENSTATIT'],
              'FE+2': ['OLIVINE(SS)','FAYALITE'],
              'MG+2': ['OLIVINE(SS)','FORSTERITE'],
              'O2':'MAGNETITE'}

We must also specify the solid solution composition of the rock, using a dictionary of dictionaries:

In [34]:
solid_solutions = {'OLIVINE(SS)':{'FORSTERITE':0.9,
                                  'FAYALITE': 0.1},
                   'ORTHOPYROXENE(SS)':{'ORTHOENSTATIT':0.8,
                                        'FERROSILITE':0.2}}

Let's generate the fluid.

In [35]:
T = 300.0 + 273.15 # in K
P = 10000.0 # in bar
dew_fluid = pyQ3.Fluid(dew_system,T,P,mineral_eq=mineral_eq, solid_solutions=solid_solutions)

/Users/simonmatthews/repos/pyQ3/pyQ3/output.py:150: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  ss_info = pd.Series()


How much Mg, Fe, and Si are dissolved in the fluid?

In [36]:
dew_fluid.elemental_comp

,element,ppm,molality
0,O,888302.7236,.5552100226E+02
1,H,111906.8956,.1110297605E+03
2,SI,78.6747,.2801255501E-02
3,FE,.1229,.2200354084E-05
4,MG,14.9290,.6142373442E-03


And what species are present in the fluid?

In [37]:
dew_fluid.aqueous_species

,species,molality,log_g,activity
0,SIO2(AQ),2.644463e-03,0.0000,2.644000e-03
1,OH-,1.015305e-03,-0.0200,9.696000e-04
2,MG+2,4.728283e-04,-0.0801,3.932000e-04
3,MG(OH)+,1.352547e-04,-0.0200,1.292000e-04
4,H2(AQ),9.695729e-05,0.0000,9.696000e-05
5,HSIO3-,7.783989e-05,-0.0200,7.433000e-05
6,SI2O4(AQ),3.639923e-05,0.0000,3.640000e-05
7,MG(HSIO3)+,6.154304e-06,-0.0200,5.877000e-06
8,H+,3.200399e-06,-0.0200,3.056000e-06
9,FE(OH)+,1.175480e-06,-0.0200,1.123000e-06


The pH?

In [38]:
dew_fluid.pH

5.5148

Is this a feasible mineral assemblage?

In [39]:
dew_fluid.mineral_saturation

,mineral,log_q_k,aff,state
0,MAGNETITE,0,0,satd
1,HEMATITE,-0.655,-1.718,NaN
2,PERICLASE,-2.37,-6.216,NaN
3,BRUCITE,-0.405,-1.062,NaN
4,FORSTERITE,-0.046,-0.12,satd
5,FAYALITE,-1,-2.623,NaN
6,CHRYSOTILE,2.104,5.519,ssatd
7,CLINOENSTATI,-0.057,-0.149,satd
8,ORTHOENSTATI,-0.097,-0.254,satd
9,PROTOENSTATI,-0.27,-0.709,NaN


Well, it looks like a fluid infiltrating the rock will react with the mineral assemblage to form some hydrated phases... This seems likely to happen in reality! This could be a good starting point for an EQ6 reaction path calculation...